In [8]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

#from config import db_password

import time

In [9]:
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [10]:
# Create a function that takes in arguments
def extract_transform_load():
    #Read in the kaggle metadata
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)

    #READ IN FILE FOR THE-NUMBERS
    #numbers_df = pd.DataFrame(f'{file_dir}/TheNumbersData.csv',encoding='unicode_escape')
    numbers_df = pd.read_csv("Resources2/TheNumbersData.csv", encoding= 'unicode_escape')
    numbers_df = numbers_df.drop(['ï»¿Number', 'Release Date', 'Domestic Gross'], axis=1)

    cols_to_check = ['Production Budget','Worldwide Gross']
    numbers_df[cols_to_check] = numbers_df[cols_to_check].replace({'Â':''}, regex=True)

    numbers_df.rename({'Movie':'original_title'},axis='columns', inplace=True)
    
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
 
    wiki_movies = [movie for movie in wiki_movies_raw
                if ('No. of episodes' not in movie)]

    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    wiki_movies_df = pd.DataFrame(clean_movies)

    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
    print(len(wiki_movies_df))
    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)

    try: 
        print(len(wiki_movies_df))
    except NameError:
        print("Skipping")
    
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    box_office = wiki_movies_df['Box office'].dropna()

    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)    

    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

        
    def parse_dollars(s):
        if type(s) != str:
            return np.nan

        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**6
            return value

        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value

        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            s = re.sub('\$|,','', s)
            value = float(s)
            return value

        else:
            return np.nan

    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)

    budget = wiki_movies_df['Budget'].dropna()    
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)

    budget = budget.str.replace(r'\[\d+\]\s*', '')
    budget[~matches_form_one & ~matches_form_two]

    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)


    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'

    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
     
    # Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] == 'True'
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    #pd.to_datetime(ratings['timestamp'], unit='s')
    #ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

    # Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
    
    # Drop unnecessary columns from the merged DataFrame.
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)
    

    # Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)

    # Call the function with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

    # Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

    # Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)    
      
    # 9. Transform and merge the ratings DataFrame.
    #rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
    #            .rename({'userId':'count'}, axis=1) \
     #           .pivot(index='movieId',columns='rating', values='count')
    #rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    #left merge to add ratings to movies_df
    #movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    #replace missing ratings with 0
    #movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
        #Cleaning numbers dataframe
    #drop unnecessary column
    
    
    
    return wiki_movies_df,movies_df, numbers_df


In [11]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '/Users/ajennings/Desktop/rutgers_bootcamp/Class_Project/data_cleaning/Resources2/'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
numbers_file = f'{file_dir}TheNumbersData.csv'

In [15]:
numbers_file.head()

,original_title,Production Budget,Worldwide Gross
0,Avengers: Endgame,"$400,000,000","$2,797,800,564"
1,Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$1,045,713,802"
2,Avengers: Age of Ultron,"$365,000,000","$1,395,316,979"
3,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$2,064,615,817"
4,Avengers: Infinity War,"$300,000,000","$2,048,359,754"


In [12]:
#Set the  variables equal to the function created above.
wiki_file, kaggle_file, numbers_file = extract_transform_load()

7305
7050


<ipython-input-10-4cfb0f306461>:81: FutureWarning: The default value of regex will change from True to False in a future version.
  budget = budget.str.replace(r'\[\d+\]\s*', '')


In [13]:
# Set the DataFrames from the return statement equal to the file names above. 
wiki_movies_df = wiki_file
movies_df = kaggle_file
numbers_df = numbers_file

In [ ]:
wiki_movies_df.head()

In [ ]:
movies_df.head()

In [16]:
numbers_df.head()

,original_title,Production Budget,Worldwide Gross
0,Avengers: Endgame,"$400,000,000","$2,797,800,564"
1,Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$1,045,713,802"
2,Avengers: Age of Ultron,"$365,000,000","$1,395,316,979"
3,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$2,064,615,817"
4,Avengers: Infinity War,"$300,000,000","$2,048,359,754"
